#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [55]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\felix\OneDrive\Studium\3. Semester\Technische Informatik\labor\gamemode_1\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [56]:
text = "This is a test document."
# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.04895174875855446, -0.039861924946308136, -0.021562796086072922, 0.009908466599881649, -0.03810402378439903, 0.012684384360909462, 0.04349454864859581, 0.07183399051427841, 0.009748565033078194, -0.006986980792135]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [58]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(file_path=pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 10000, chunk_overlap = 20)
pages_chunked = splitter.split_documents(pages)

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

def clean_and_create_document(chunk):
    cleaned_text = clean_text(chunk.page_content)
    return Document(page_content=cleaned_text, metadata=chunk.metadata)


pages_chunked_cleaned = [clean_and_create_document(chunk) for chunk in pages_chunked]


print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [59]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [60]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 501


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [62]:
from langchain_chroma import Chroma
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("collection_name")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client=client,
    collection_name="collection_name",
    embedding_function=embedding_model,
    )

In [90]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

vector_db_from_client.add_documents(documents=pages_chunked_cleaned[:50], id = uuids)


#uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
#vector_db_from_client.add_texts(pages_chunked_cleaned, id = uuids)

['e3eb6a96-6739-4966-9e97-188a842ca033',
 '6329692c-f436-41a6-a8dd-893ed09c7d77',
 '4de1a3a8-2341-4d43-8e2c-b56f0030b822',
 '8a4077f6-27f8-46a1-bfbb-1bf70565f74f',
 '312f0e44-c5d8-4a0a-8d75-707602c6bd23',
 '43f892a9-1ac7-4ccc-8362-be49415f1cf8',
 '1e8cc2e0-cf70-469c-8777-a81e1637d893',
 '3ba9fd43-ed30-4cf1-9f27-b47f3c0cbe9c',
 '5436a23a-4a8d-4005-9d42-e26c4a54a829',
 '1f6e8357-2f99-4a96-9dc1-dd8fe3a32a25',
 'e3697f20-e173-4987-8320-a93bd15159bc',
 '83568122-1526-4c29-ab51-99dbf559f9b7',
 'a7c5d227-c8cb-43ed-a3f2-2fa3c81db835',
 '8a9a4f77-39b6-46a2-91d0-2f3d34358001',
 'ca174bb2-adf7-49d9-9b85-72e3b687697c',
 '93ffc1f4-87f2-4b33-a938-71ba23421ed8',
 '8215a0c1-557b-47d0-9f21-054eba370bf6',
 'f49cd33e-02b9-4f8d-9424-5085fbdca9a4',
 'cfcf3163-7d0e-432b-96dc-ca51f6f3c829',
 'ff7f9921-d682-4ac9-9547-4d126e1ad450',
 '05996117-bcb6-4192-bbd9-2d6194982b99',
 '6df2b80e-7140-45dd-b3d4-194218a124cb',
 'a0f2720b-2740-4766-97f0-4897fad8c59f',
 '965e9cf6-162d-4ba6-9fbb-8803df64d0e0',
 'f249f136-685e-

In [91]:
client.count_collections()

1

In [92]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [93]:
search_query = str("Types of Machine Learning Systems")

results = vector_db_from_client.similarity_search(
    "search_query",
    k=3,
    )

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Exception: {"error":"InvalidArgumentError","message":"Expected where to have exactly one operator, got {}"}